<a href="https://colab.research.google.com/github/rossann9119/dw_matrix_car/blob/master/day3_simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [7]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


Wczytywanie Danych


In [8]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [9]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

Dummy Model


In [10]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [12]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)

mean_absolute_error(y, y_pred)

39465.934630440985

In [14]:
df['price_currency'].value_counts(normalize=True)

PLN    0.998084
EUR    0.001916
Name: price_currency, dtype: float64

In [15]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [16]:
df = df[ df['price_currency'] != 'EUR']
df.shape

(106290, 155)

Features

In [18]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
SUFFIX_CAT = '_cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [24]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats =  [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [25]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368328

In [27]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)


Weight,Feature
0.2533 ± 0.0040,param_napęd_cat
0.2008 ± 0.0047,param_faktura-vat_cat
0.1943 ± 0.0088,param_stan_cat
0.1423 ± 0.0086,param_rok-produkcji_cat
0.0629 ± 0.0047,param_moc_cat
0.0424 ± 0.0014,feature_kamera-cofania_cat
0.0412 ± 0.0008,param_skrzynia-biegów_cat
0.0286 ± 0.0037,param_marka-pojazdu_cat
0.0191 ± 0.0022,param_pojemność-skokowa_cat
0.0163 ± 0.0005,feature_bluetooth_cat
